<a href="https://colab.research.google.com/github/maharani032/sentimen/blob/main/Preprocessing.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install sastrawi
!pip install preprocessor

In [14]:
import pandas as pd
import preprocessor as p
from textblob import TextBlob
from nltk.stem import PorterStemmer 
from nltk.tokenize import word_tokenize
import numpy as np
import re
from Sastrawi.StopWordRemover.StopWordRemoverFactory import StopWordRemoverFactory, StopWordRemover, ArrayDictionary
import nltk
nltk.download('stopwords')

from nltk.corpus import stopwords
from Sastrawi.Stemmer.StemmerFactory import StemmerFactory
# import tokenize
from nltk.tokenize import TweetTokenizer

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
data = pd.read_excel("/content/drive/MyDrive/Skripsi/Database baru/Data tweet CHATGPT dan label v2.xlsx")
data.info()

In [4]:
# remove mention
def remove_mention(tweet,pattern_regex):
    r=re.findall(pattern_regex,tweet)
    for i in r:
        tweet=re.sub(i,"",tweet)
    return tweet

In [5]:
def remove_http(tweet):
    tweet=" ".join(re.sub("(@[A-Za-z0-9]+)|([^0-9A-Za-z \t])|(\w+:\/\/\S+)"," ",tweet).split())
    return tweet

In [6]:
def removeHastag(tweet):
#     hastag dolar sign
    tweet=re.sub(r'\$\w*','',tweet)
#     retweet lama
    tweet=re.sub(r'^RT[\s]+','',tweet)
#     hastag
    tweet=re.sub(r'#', '',tweet)
#     hapus angka
    tweet=re.sub('[0-9]+','',tweet)
    return tweet

In [ ]:
def case_folding(tweet):
    return tweet.casefold()
data['remove_mention']=np.vectorize(remove_mention)(data['tweet']," *RT* | *@[\w]*")
data['remove_http']=data['remove_mention'].apply(lambda x:remove_http(x))
data['remove_hastag']=data['remove_http'].apply(lambda x:removeHastag(x))
data['case_folding']=data['remove_hastag'].apply(lambda x:case_folding(x))
data.drop_duplicates(subset='case_folding',keep='first',inplace=True)
data.head(5)

In [ ]:
stopwords_indonesia=stopwords.words('indonesian')
# import sastrawi

In [10]:
more_stopword = [
    'yg','dgn','kan','huh','bruh','xd','xf','hahaha','nya','wkwkwkkwkwk','akwkaskaksawska','wkwkw','wkwkkwwkk'
    'kok','kyk','dong','donk','yah','tuh','si','siii','wkwk','wkwkwk','ini','tp','utk','sj','pd','gw','gua','gwe',
    'wow','wowwwwwah','icymi','ni','coy','coii','isenkisenk','dg','pdhl','aja','tadi','krn','tak',
    'aja','sbb','kuy','se','skrg','yep','aja','as','yaa','jadinya','aja','coba','tibatiba','shit','knp','jdi','dmn','kyk'
    'sih','bang','oke','nah','bgt','km','ttg','dlm','aaa','kang','hehe','wes','you','doang','wkkw','ong','sm','he','yeee'
    ,'emg','kak','gan','woy','dm','hi','kakk','min','di','noh','gais','lah','xfxf','nak','bro','x','ahhh','gasss','hmmm','sat','set','yukkkkk'
    ,'smh','eh','ni','laaah',' aihihi','fafifu','akwkaskaksawska ','xixi','kah','tbtb','bg','jg','pas','w','jga','cm','hiks','mennn','sii','sy','aku',
    ]

In [16]:
stop_factory = StopWordRemoverFactory().get_stop_words()
stopword=stop_factory+more_stopword+stopwords_indonesia
dictionary=ArrayDictionary(stopword)
str=StopWordRemover(dictionary)
factory=StemmerFactory()
stemmer=factory.create_stemmer()

In [17]:
def clean_tweets(tweet):
#     tokenize tweets
    tokenizer=TweetTokenizer(preserve_case=False,strip_handles=True,reduce_len=True)
    tweet_token=tokenizer.tokenize(tweet)
    
    tweet_clean=[]
    for word in tweet_token:
        if(word not in stopword):
            stem_word=stemmer.stem(word)
            tweet_clean.append(stem_word)
    return tweet_clean

In [18]:
def tweet(tweet):
    return " ".join(tweet)

In [19]:
data['clean_tweet']=data['case_folding'].apply(lambda x:clean_tweets(x))
data['clean tweet']=data['clean_tweet'].apply(lambda x:tweet(x))


In [20]:
newData=pd.DataFrame()
newData['date']=data['date']
newData['username']=data['username']
newData['tweet']=data['tweet']
newData['clean tweet']=data['clean tweet']
newData['label']=data['label']
filename='processing-lengkap.xlsx'
newFilename='preprocessing-sudah.xlsx'
data.to_excel('/content/drive/MyDrive/Skripsi/Database baru/'+filename, index=False, encoding='utf-8')
newData.to_excel('/content/drive/MyDrive/Skripsi/Database baru/'+newFilename, index=False, encoding='utf-8')

/usr/local/lib/python3.9/dist-packages/pandas/util/_decorators.py:211: FutureWarning: the 'encoding' keyword is deprecated and will be removed in a future version. Please take steps to stop the use of 'encoding'
  return func(*args, **kwargs)
